<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/few_shot_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 7.6MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 655kB 22.8MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 8.2MB 18.8MB/s 
     |████████████████████████████████| 20.1MB 222kB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 512kB 57.6MB/s 
     |████████████████████████████████| 860kB 40.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 6.7MB 27.5MB/s 
     |████████████████████████████████| 2.1MB 48.5MB/s 
     |████████████████████████████████| 1.5MB 21.9MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 10.4MB 46.1MB/s 
     |████████████████████████████████| 7.3MB 18.4MB/s 
    

In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 512kB 34.8MB/s 
     |████████████████████████████████| 3.8MB 38.6MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=38dc345983e0e2c2948a5bdc9f3db9cbf9cfa3b604603e9780a00ee741262f72
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [3]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [4]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-iveomp2e
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-iveomp2e
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=1c249827f4da49d41083e47865c2b205cdb7892d998f3e2a77f76a2d05c057f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-l2ncd8ey/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [5]:
# # !rm few_test.txt
# !rm checkpoint  model.data-00000-of-00001  model.meta   tag.dict  train.txt logs model.index sample_data test.txt valid.txt
# !ls

In [6]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/few_test.txt

--2021-04-11 21:56:22--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/few_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45016 (44K) [text/plain]
Saving to: ‘few_test.txt’

few_test.txt        100%[===================>]  43.96K  --.-KB/s    in 0.007s  

2021-04-11 21:56:22 (5.82 MB/s) - ‘few_test.txt’ saved [45016/45016]



In [7]:
!mv few_test.txt train.txt

In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/test.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/valid.txt

--2021-04-11 21:56:24--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5188 (5.1K) [text/plain]
Saving to: ‘test.txt’

test.txt            100%[===================>]   5.07K  --.-KB/s    in 0s      

2021-04-11 21:56:24 (57.8 MB/s) - ‘test.txt’ saved [5188/5188]

--2021-04-11 21:56:24--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [9]:
# !cp train.txt test.txt
# !cp train.txt valid.txt
!ls

sample_data  test.txt  train.txt  valid.txt


In [10]:

import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

In [11]:
# ner_config
# open('train.txt').readlines()

In [12]:
ner_model = train_model(ner_config, download=True)

2021-04-11 21:56:26.611 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:11<00:00, 3.46MB/s]
2021-04-11 21:59:39.399 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models


Exception: ignored

In [ ]:
ner_model(["какое-то не лекарство", "пью леонезит 1 месяц", "варкрафт 2 интересная игра"])

In [ ]:
nanika = ner_model(['Паксил норм. Принимал около года, щас перерыв. \
  Синдром отмены не то чтобы совсем отсутствует, но мне особых непрятностей не доставил. \
  Еще тироксин в качестве антидепрессанта хорош, хотя формально показания у него другие.'])

In [ ]:
list(zip(nanika[0][0], nanika[1][0]))